(solph-linear)=

# Linear oemof-solph heat pump model

In the first section we will implement a heat pump with the Carnot COP method and a linear efficiency factor. For that
we are going to build the system shown in {numref}`Systemoverview` using `oemof-solph`.

## Build the model

### General setup

First we are building the components of the system. We do that, by importing the library and create the `Bus` components
and the electricity grid connection first.

In [ ]:
import oemof.solph as solph


b_electricity = solph.Bus(label="electricity")
b_heat_35C = solph.Bus(label="heat 35C")

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={b_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

### Storage

Next we add our storage to the system. The storage is connected to the heat bus and has a specified loss rate of 2 %.
The nominal capacity of the storage will be 8.7 kWh.

In [ ]:
thermal_storage = solph.components.GenericStorage(
    label='thermal storage',
    inputs={b_heat_35C: solph.Flow()},
    outputs={b_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

### Heat demand

For our heat demand we need a timeseries. We have prepared the data and can load them with a predefined function in
this tutorial.

In [ ]:
from utilities import load_input_data


input_data = load_input_data().head(24*2)
demand = input_data["Heat load (kW)"][:-1]

demand

That demand time series can be connected to a `Sink` component.

In [ ]:
heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={b_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)

### Heat Pump

Now, we want to add an air-source heat-pump. From a quick research we find the following example {cite}`viessmann2023`:

* Rated heating output (kW) at operating point A7/W35: 9.1 kW
* Coefficient of performance COP A7/W35: 4.9

From the datasheet we know what the COP of the heat pump is for a specific set of temperatures at nominal load, i.e.

- an ambient temperature of 7 °C and
- heat production temperature of 35 °C.

However, we do not know, the COP at ambient temperature levels different from 7 °C. With the efficiency factor from
eq. {eq}`cop-heat-pump-carnot-and-efficiency` and the definition of the Carnot COP (eq. {eq}`carnot-cop-heat-pump`) we
can find the actual COP of the heat pump **under the assumption, that the efficiency factor is a constant value**. We
can plot the data 

In [ ]:
datasheet_cop = 4.9

carnot_cop_7_35 = (35+273.15) / (35-7)
cpf_7_35 = datasheet_cop / carnot_cop_7_35

input_data["cpf COP 7 -> 35"] = cpf_7_35 * (35+273.15) / (35 - input_data["Ambient temperature (°C)"])

In [ ]:
from myst_nb import glue
from utilities import TTD_HEAT_EXCHANGERS, HEATING_SYSTEM_FEED_TEMP, AMBIENT_TEMP_NOMINAL
glue("heat-exchanger-ttd", TTD_HEAT_EXCHANGERS, display=False)
glue("evaporation-temperature", AMBIENT_TEMP_NOMINAL - TTD_HEAT_EXCHANGERS, display=False)
glue("condensation-temperature", HEATING_SYSTEM_FEED_TEMP + TTD_HEAT_EXCHANGERS, display=False)

However, note that we have to use the maximum and minimum temperature of the process when working with the Carnot COP.
The figure {numref}`heat-pump-black-box-vs-process` shows two different representations of the heat pump: One as a black
box and one as component based process. In the black box process, we only see the temperature levels of the air and the
water system. In the actual heat pump heat is transferred from the air to the working fluid and from the working fluid
to the water system. This cannot happen without thermodynamic losses, meaning that the temperature of the working fluid
must be lower than the air temperature when evaporating and the temperature must be higher when condensing and providing
heat to the water system. This however changes our reference temperature levels for the Carnot COP, as the maximum and
minimum temperatures of the process are not the air and water temperatures but refer to the working fluid.

```{figure} /figures/heat_pump_blackbox_comparison.svg
---
alt: Heat pump black box compared to actual heat pump process
name: heat-pump-black-box-vs-process
---
Heat pump black box compared to actual heat pump process
```

Since we do not have any internal data of the heat pump, we have to make an assumption to determine the Carnot COP in
this way. For both heat exchangers we assume a temperature difference of {glue:text}`heat-exchanger-ttd` K, therefore
a condensation temperature of {glue:text}`condensation-temperature` °C and an evaporation temperature of
{glue:text}`evaporation-temperature` °C. This changes our Carnot COP, and given, that we want to have the same COP as in
our first assessment, the efficiency factor.

In [ ]:
import numpy as np

carnot_cop_2_40 = (40+273.15) / (40-2)
cpf_2_40 = datasheet_cop / carnot_cop_2_40

input_data["cpf COP 2 -> 40"] = cpf_2_40 * (40 + 273.15) / (40 - input_data["Ambient temperature (°C)"] + 5)


temperature_range = np.arange(-10, 21)
cop_7_35 = cpf_7_35 * (35 + 273.15) / (35 - temperature_range)
cop_2_40 = cpf_2_40 * (40 + 273.15) / (40 - temperature_range + 5)

We can have a look at the difference between both approaches in {numref}`cop-reference-temperature-variants`. The left
subplot shows the COP with both approaches over a range of the ambient temperature between -10 and 20 °C. There we can
see, that both methods yield the same outcome for 7 °C, i.e. 4.9. For higher temperatures the variant with 7/35 as
reference temperatures is higher, for the lower temperatures the variant with 2/40 as reference temperatures has a
higher COP. The right subplot shows the COP time series for our application example. Since the ambient temperature is
mostly higher than 7 °C, the implementation of the 7/35 variant will have higher COP most of the time.

```{glue:figure} cop-reference-temperature-variants
:name: "cop-reference-temperature-variants"

Comparison of the COP of the heat pump calculated with 7/35 and 2/40 as reference temperature values.
```

In [ ]:
import matplotlib.pyplot as plt


fig, ax  = plt.subplots(1, 2, sharey=True, figsize=(12, 5))

ax[0].plot(temperature_range, cop_7_35, label="COP 7;35")
ax[0].plot(temperature_range, cop_2_40, label="COP 2;40")

ax[0].set_ylabel("COP")
ax[0].set_xlabel("Ambient temperature")

ax[1].plot(input_data["cpf COP 7 -> 35"])
ax[1].plot(input_data["cpf COP 2 -> 40"])

ax[0].legend()

_ = ax[1].set_xlabel("timestep")

plt.close()

In [ ]:
glue("cop-reference-temperature-variants", fig, display=False)

With this information we can create the heat pump, we will use the 2/40 temperature reference.

In [ ]:
hp_thermal_power = 9.1  # kW

cop = input_data["cpf COP 2 -> 40"][:-1]

In [ ]:
heat_pump = solph.components.Transformer(
    label="heat pump",
    inputs={b_electricity: solph.Flow()},
    outputs={b_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        b_electricity: 1 / cop,
        b_heat_35C: 1,
    },
)

### Optimize the energy system

Finally, with all components in place we can create the energy system, add the components and run the optimization.

In [ ]:
es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

es.add(b_electricity, b_heat_35C)
es.add(electricity_grid, heat_demand)
es.add(heat_pump)
es.add(thermal_storage)

model = solph.Model(energysystem=es)
model.solve()
results = solph.processing.results(model)

After running the optimization we extract the results and create a figure for the duration curve of the heat pump
operation and the demand time series as well as the dispatch of the heat pump and the storage.
{numref}`results-solph-simple` shows the results of our first simple implementation. The heat pump is mostly operated in
a way to provide the heat demand directly without using the storage as the storage losses make that option unattractive.
Only in a single time step the heat pump feeds additional heat to the storage to make use of a advantageous COP for the
production of the heat. A total of {glue}`results-electricity-simple` kWh of electricity is consumed.

```{glue:figure} results-solph-simple
:name: "results-solph-simple"

Results of the simple heat pump setup.
```

```{tip}
We have included a function `sumarise_solph_results`, which extracts the dispatch of the heat pump, the demand and the
storage filling level to create comparable plots in all sections.
```

```{literalinclude} utilities.py
:language: python
:pyobject: sumarise_solph_results
```

In [ ]:
from utilities import sumarise_solph_results

fig, electricity_total = sumarise_solph_results(results)
plt.close()

In [ ]:
glue("results-solph-simple", fig, display=False)
glue("results-electricity-simple", round(electricity_total, 2), display=False)